In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key not set


In [ ]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [ ]:
class ConversationManager:
    def __init__(self):
        self.conversation_history = []
        self.participants = {}
    
    def add_participant(self, name, chatbot):
        """Add a model to the conversation"""
        self.participants[name] = chatbot
    
    def add_message(self, speaker, message):
        """Add a message to the shared conversation history"""
        self.conversation_history.append({
            "speaker": speaker,
            "role": "assistant" if speaker in self.participants else "user",
            "content": message
        })
        
    def get_context_for_model(self, model_name):
        """Create context appropriate for the given model"""
        # Convert the shared history to model-specific format
        messages = []
        for msg in self.conversation_history:
            if msg["speaker"] == model_name:
                messages.append({"role": "assistant", "content": msg["content"]})
            else:
                messages.append({"role": "user", "content": msg["content"]})
        return messages
    
    def run_conversation(self, starting_message, turns=3, round_robin=True):
        """Run a multi-model conversation for specified number of turns"""
        current_message = starting_message
        models = list(self.participants.keys())
        
        # Add initial message
        self.add_message("user", current_message)
        
        for _ in range(turns):
            for model_name in models:
                # Get context appropriate for this model
                model_context = self.get_context_for_model(model_name)
                
                # Get response from this model
                chatbot = self.participants[model_name]
                response = chatbot.generate_response(model_context)
                
                # Add to conversation history
                self.add_message(model_name, response)
                
                print(f"{model_name}:\n{response}\n")
                
                if not round_robin:
                    # If not round-robin, use this response as input to next model
                    current_message = response

In [ ]:
class ChatBot:
    def __init__(self, model_name, system_prompt, **kwargs):
        self.model_name = model_name
        self.system_prompt = system_prompt
        self.api_key = kwargs.get('api_key', None)
        self.base_url = kwargs.get('base_url', None)
        
    def generate_response(self, messages):
        """Generate a response based on provided messages without storing history"""
        # Prepare messages including system prompt
        full_messages = [{"role": "system", "content": self.system_prompt}] + messages
        
        try:
            if "claude" in self.model_name.lower():
                # Format messages for Claude API
                claude_messages = [m for m in messages if m["role"] != "system"]
                response = anthropic.Anthropic().messages.create(
                    model=self.model_name,
                    system=self.system_prompt,
                    messages=claude_messages,
                    max_tokens=200,
                )
                return response.content[0].text
                
            else:
                # Use OpenAI API (works for OpenAI, Gemini via OpenAI client, etc)
                openai_client = OpenAI(api_key=self.api_key, base_url=self.base_url)
                response = openai_client.chat.completions.create(
                    model=self.model_name,
                    messages=full_messages,
                    max_tokens=200,
                )
                return response.choices[0].message.content
            
        except Exception as e:
            return f"Error: {str(e)}"

In [ ]:
# Initialize models
gpt_bot = ChatBot("gpt-4o-mini", "You are witty and sarcastic.")
claude_bot = ChatBot("claude-3-haiku-20240307", "You are thoughtful and philosophical.")

model_name = "qwen2.5:1.5b"
system_prompt = "You are a helpful assistant that is very argumentative in a snarky way."
kwargs = {
    "api_key": "ollama",
    "base_url": 'http://localhost:11434/v1'
}
qwen = ChatBot(model_name, system_prompt, **kwargs)

# Set up conversation manager
conversation = ConversationManager()
conversation.add_participant("GPT", gpt_bot)
conversation.add_participant("Claude", claude_bot)
conversation.add_participant("Qwen", qwen)

# Run a multi-model conversation
conversation.run_conversation("What's the most interesting technology trend right now?", turns=2)